In [ ]:
%pip install -U "geemap[workshop]"

In [2]:
import ee
import geemap
import geemap.colormaps as cm
from geemap import geojson_to_ee, ee_to_geojson

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
import json

# City Municipal Districs
with open("./mo.geojson") as file:
    districts = geojson_to_ee(json.load(file))

In [ ]:
def ndvi(img: ee.Image):
    ndvi = img.normalizedDifference(["B5", "B4"]).rename("NDVI")
    return img.addBands(ndvi)

collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

In [ ]:
years = range(2021, 2025)
summers = [[f"{year}-06-01", f"{year}-08-31"] for year in years]
filters = [ee.Filter.date(*summer) for summer in summers]
allSeasonsFilter = ee.Filter.Or(filters)

In [ ]:
# Number of images

def add_year(img: ee.Image):
    date = ee.Date(img.get("DATE_ACQUIRED"))
    return img.set("YEAR", date.get("year"))

images = collection.filterBounds(districts.geometry()) \
    .filter(allSeasonsFilter) \
    .filter(ee.Filter.lt("CLOUD_COVER", 10)) \
    .map(ndvi) \
    .select(['B2','B3','B4','B5','NDVI']) \
    .map(add_year)

print(images.size().getInfo())

In [ ]:
import pandas as pd

year_acquired=images.aggregate_array("YEAR").getInfo()
year_acquired = pd.DataFrame(year_acquired, columns=["year"])
year_acquired["year"].value_counts().plot(kind='bar')

In [ ]:
# years = range(2019, 2025)
years = [2019, 2024]
images = {}
for year in years:
    summer = [f"{year}-06-01", f"{year}-08-31"]
    filter = ee.Filter.date(*summer)
    seasonCollection = collection.filterBounds(districts.geometry()) \
        .filter(filter) \
        .filter(ee.Filter.lt("CLOUD_COVER", 4)) \
        .select(['B2','B3','B4','B5','B6']) \
        

    print(f"{year} : {seasonCollection.size().getInfo()}")
    min_image = seasonCollection.min()
    ndvi = min_image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi = min_image.normalizedDifference(['B3', 'B6']).rename('MNDWI')
    image = min_image.addBands(ndvi).addBands(mndwi)
    images.update({year: image})



In [ ]:

mask = images[years[-1]].select(['MNDWI']).lte(0.5)

ndvis = [mask]
for year in years:
    name = f"NDVI{year}"
    ndvi = images[year].select(['NDVI']).mask(mask).rename(name)
    ndvis.append(ndvi)

ndvi_image = ee.Image.cat(ndvis)

name_first = f"NDVI{years[0]}"
name_last = f"NDVI{years[-1]}"

ndvi_diff = ndvi_image.expression(f'b("{name_last}")-b("{name_first}")').rename('DIFF')
ndvi_diff_per = ndvi_image.expression(f'(b("{name_last}")-b("{name_first}"))/((b("{name_last}")+b("{name_first}"))/2)').rename('DIFF_PER')

ndvi_image = ndvi_image.addBands(ndvi_diff).addBands(ndvi_diff_per)

ndvi_image.bandNames().getInfo()

In [ ]:
rgb_vis = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 0.3}
ndvi_change_vis = {"min": -1, "max": 1, "palette": ["red", "white", "green"]}
ndvi_vis = {"bands": ["NDVI"], "min": 0, "max": 1, "palette": cm.palettes.ndvi}

map = geemap.Map(center=[55.71782880151228, 37.62268066406251], zoom=8, height=1000)

# for year, image in images.items():
#     map.addLayer(image, rgb_vis, f"{year}")
    
for year, image in images.items():
    map.addLayer(image, ndvi_vis, f"{year} NDVI")

map.addLayer(ndvi_image, {"bands": ["DIFF"], **ndvi_change_vis}, f"NDVI DIFF")
# map.addLayer(ndvi_image, {"bands": ["DIFF_PER"], **ndvi_change_vis}, f"NDVI DIFF (%)")
map.addLayer(districts.style(**{'color': '5b5b5b', 'width': 0.2, 'fillColor': '00000000'}), {}, "Moscow Districts")
map

In [ ]:
districts_ndvi = districts
for year in years:
    districts_ndvi = ndvi_image.select([f'NDVI{year}']) \
        .reduceRegions(
            collection = districts_ndvi,
            reducer=ee.Reducer.mean().setOutputs([f"NDVI{year}"]),
            scale=30,
            crs='EPSG:4326'
        )
    

districts_ndvi = ndvi_image.select(['DIFF']) \
    .reduceRegions(
        collection = districts_ndvi,
        reducer=ee.Reducer.mean().setOutputs([f"NDVI_DIFF"]),
        scale=30,
        crs='EPSG:4326'
    )


In [ ]:
gdf = geemap.ee_to_gdf(districts_ndvi)
gdf = gdf.sort_values(by='NDVI_DIFF').reset_index(drop=True)
print(gdf[['NAME', 'NAME_AO', 'NDVI_DIFF']].to_markdown())


In [ ]:
bands = []
for year in years:
    name = f"NDVI{year}"
    bands.append(name)

bands.append("DIFF")

task = ee.batch.Export.image.toDrive(
    image=ndvi_image.select(bands),
    description=f'NDVI-{years[0]}-{years[-1]}',
    scale=30, 
    crs='EPSG:4326',
    maxPixels = 892227573612
)
task.start()

In [ ]:
print(task.status())